<a href="https://colab.research.google.com/github/ysfArslan1/tf-idf/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
import os
from nltk.corpus import stopwords
import nltk
import re
import string
import spacy


stop_words =  set(stopwords.words('english'))
stemmer = nltk.porter.PorterStemmer()#‘s’, ‘es’, ‘ed’, ‘ing’, ‘ly’
nlp_spacy = spacy.load('en',parse=True,tag=True, entity=True)
nlp = spacy.load('en_core_web_sm')
#print(stop_words)

path= "/content/drive/MyDrive/kodlamalar/okul_projesi/nlp/Dataset/manuscripts/"
pdfler = os.listdir(path)
veriler = []
for pdf in pdfler:
    try:
        veri= {
            "isim":"",
        }
        veri.update({"isim" : pdf})
        dosya = open(path+pdf,"r")
        for metin in dosya:
            veri.update({"metin":metin})
        
        dosya.close()
        veriler.append(veri)
    except:
        print( "hata")

pdf_sayisi = len(veriler)
#print(pdf_sayisi)

last_word = []

#print(dir(veriler[0][0]))
for i in range(pdf_sayisi):
    metin = veriler[i]["metin"]
    metin = metin.lower()
    metin = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", metin)
    metin = re.sub(r'[^a-zA-z.,!?/:;\"\'\s]' , ' ', metin)
    metin = ''.join([c for c in metin if c not in string.punctuation])
    metin = ' '.join([stemmer.stem(word) for word in metin.split()])
    #metin = nlp(metin)
    #metin = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.metin for word in metin])
    metin = metin.split(" ")
    not_stop_worlds =[]
    for w in metin:
        if w not in stop_words:
            last_word.append(w)
            not_stop_worlds.append(w)
    
    veriler[i].update({"not_stop_worlds":not_stop_worlds})

uniqe_words = set(last_word)
dict = dict.fromkeys(uniqe_words, 0)


#print(dict)
print(len(dict))


12410


In [8]:
print(veriler)

[{'isim': '1605.06526.txt', 'metin': 'massless and massive higher spins from anti-de sitter space waveguide higgs mechanism to massive higher-spin gauge fields is an outstanding openproblem. we investigate this issue in the context of kaluza-kleincompactification. starting from a free massless higher-spin field in$(d+2)$-dimensional anti-de sitter space and compactifying over a finiteangular wedge, we obtain an infinite tower of heavy, light and masslesshigher-spin fields in $(d+1)$-dimensional anti-de sitter space. all massivehigher-spin fields are described gauge invariantly in terms of st\\"ueckelbergfields. the spectrum depends on the boundary conditions imposed at both ends ofthe wedges. we obseved that higher-derivative boundary condition is inevitablefor spin greater than three. for some higher-derivative boundary conditions,equivalently, spectrum-dependent boundary conditions, we get a non-unitaryrepresentation of partially-massless higher-spin fields of varying depth. wepresen

In [9]:

for i in range(pdf_sayisi):
    dict_1 = dict
    for word in veriler[i]["not_stop_worlds"]:
        dict_1[word] += 1
    veriler[i].update({"world_count":dict_1})


In [10]:
print(veriler[0])

{'isim': '1605.06526.txt', 'metin': 'massless and massive higher spins from anti-de sitter space waveguide higgs mechanism to massive higher-spin gauge fields is an outstanding openproblem. we investigate this issue in the context of kaluza-kleincompactification. starting from a free massless higher-spin field in$(d+2)$-dimensional anti-de sitter space and compactifying over a finiteangular wedge, we obtain an infinite tower of heavy, light and masslesshigher-spin fields in $(d+1)$-dimensional anti-de sitter space. all massivehigher-spin fields are described gauge invariantly in terms of st\\"ueckelbergfields. the spectrum depends on the boundary conditions imposed at both ends ofthe wedges. we obseved that higher-derivative boundary condition is inevitablefor spin greater than three. for some higher-derivative boundary conditions,equivalently, spectrum-dependent boundary conditions, we get a non-unitaryrepresentation of partially-massless higher-spin fields of varying depth. wepresent

In [11]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [12]:
for i in range(pdf_sayisi):
    veri =computeTF(veriler[i]["world_count"],uniqe_words)
    veriler[i].update({"tf-idf":veri})



In [13]:

print(veriler[i]["tf-idf"])

{'le': 0.00024174053182917003, 'willmoreenergi': 8.058017727639001e-05, 'oflinear': 0.00024174053182917003, 'thedevelop': 8.058017727639001e-05, 'nonlinearelectromagnet': 8.058017727639001e-05, 'bodyha': 8.058017727639001e-05, 'direct': 0.004834810636583401, 'dray': 8.058017727639001e-05, 'theseresult': 0.00016116035455278002, 'themechan': 8.058017727639001e-05, 'ofhypersurfac': 8.058017727639001e-05, 'solutionscorrespond': 0.00016116035455278002, 'dust': 0.00104754230459307, 'institut': 8.058017727639001e-05, 'repres': 0.00330378726833199, 'anaverag': 8.058017727639001e-05, 'themodern': 8.058017727639001e-05, 'toth': 0.0024979854955680903, 'r': 0.019016921837228043, 'tag': 0.00016116035455278002, 'fullygener': 8.058017727639001e-05, 'sitterequ': 8.058017727639001e-05, 'lema': 0.00048348106365834006, 'theseexcit': 8.058017727639001e-05, 'ln': 0.00016116035455278002, 'overth': 8.058017727639001e-05, 'admit': 0.0038678485092667205, 'eitherin': 8.058017727639001e-05, 'physicist': 0.000564

In [14]:
for i in range(pdf_sayisi):
    veriler[i].pop("metin")

In [15]:
print(veriler[0]["isim"])
print(veriler[0]["not_stop_worlds"])
print(veriler[0]["world_count"])
print(veriler[0]["tf-idf"])

1605.06526.txt
['massless', 'massiv', 'higher', 'spin', 'anti', 'de', 'sitter', 'space', 'waveguid', 'higg', 'mechan', 'massiv', 'higher', 'spin', 'gaug', 'field', 'outstand', 'openproblem', 'investig', 'thi', 'issu', 'context', 'kaluza', 'kleincompactif', 'start', 'free', 'massless', 'higher', 'spin', 'field', 'dimension', 'anti', 'de', 'sitter', 'space', 'compactifi', 'finiteangular', 'wedg', 'obtain', 'infinit', 'tower', 'heavi', 'light', 'masslesshigh', 'spin', 'field', 'dimension', 'anti', 'de', 'sitter', 'space', 'massivehigh', 'spin', 'field', 'describ', 'gaug', 'invariantli', 'term', 'st', 'ueckelbergfield', 'spectrum', 'depend', 'boundari', 'condit', 'impos', 'end', 'ofth', 'wedg', 'obsev', 'higher', 'deriv', 'boundari', 'condit', 'inevitablefor', 'spin', 'greater', 'three', 'higher', 'deriv', 'boundari', 'condit', 'equival', 'spectrum', 'depend', 'boundari', 'condit', 'get', 'non', 'unitaryrepresent', 'partial', 'massless', 'higher', 'spin', 'field', 'vari', 'depth', 'wepres'

In [16]:
def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [17]:
#idfs = computeIDF([numOfWordsA, numOfWordsB])

In [18]:
gerekli_veri = []
for i in range(pdf_sayisi):
    gerekli_veri.append(veriler[i]["world_count"])

In [19]:
idfs = computeIDF(gerekli_veri)

In [20]:
print(idfs)

{'le': 0.0, 'willmoreenergi': 0.0, 'oflinear': 0.0, 'thedevelop': 0.0, 'nonlinearelectromagnet': 0.0, 'bodyha': 0.0, 'direct': 0.0, 'dray': 0.0, 'theseresult': 0.0, 'themechan': 0.0, 'ofhypersurfac': 0.0, 'solutionscorrespond': 0.0, 'dust': 0.0, 'institut': 0.0, 'repres': 0.0, 'anaverag': 0.0, 'themodern': 0.0, 'toth': 0.0, 'r': 0.0, 'tag': 0.0, 'fullygener': 0.0, 'sitterequ': 0.0, 'lema': 0.0, 'theseexcit': 0.0, 'ln': 0.0, 'overth': 0.0, 'admit': 0.0, 'eitherin': 0.0, 'physicist': 0.0, 'infinitenumb': 0.0, 'stationar': 0.0, 'pronounc': 0.0, 'listen': 0.0, 'andtwo': 0.0, 'israel': 0.0, 'integerinto': 0.0, 'aphoton': 0.0, 'gyroscop': 0.0, 'unsuccessfullyaddress': 0.0, 'bell': 0.0, 'threeregion': 0.0, 'pig': 0.0, 'horizonspacetim': 0.0, 'superradiantli': 0.0, 'quickli': 0.0, 'beenwritten': 0.0, 'call': 0.0, 'vmcg': 0.0, 'mir': 0.0, 'sinerti': 0.0, 'laplac': 0.0, 'thediscuss': 0.0, 'roughlycorrespond': 0.0, 'textit': 0.0, 'isinvestig': 0.0, 'octob': 0.0, 'applythi': 0.0, 'dyon': 0.0, 'cou

In [21]:
def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [22]:
"""
tfidfA = computeTFIDF(tfA, idfs)
tfidfB = computeTFIDF(tfB, idfs)
df = pd.DataFrame([tfidfA, tfidfB])
"""

'\ntfidfA = computeTFIDF(tfA, idfs)\ntfidfB = computeTFIDF(tfB, idfs)\ndf = pd.DataFrame([tfidfA, tfidfB])\n'

In [23]:
for i in range(pdf_sayisi):
  deger = computeTFIDF(veriler[i]["tf-idf"], idfs)
  veriler[i].update({"tf-idf":deger})

In [24]:
db=[]
for i in range(pdf_sayisi):
  db.append(veriler[i]["tf-idf"])

In [25]:
import pandas as pd
df = pd.DataFrame(db)

In [26]:
df.head()

,le,willmoreenergi,oflinear,thedevelop,nonlinearelectromagnet,bodyha,direct,dray,theseresult,themechan,...,hierarch,characteristiciniti,pullback,thread,fieldin,andblack,therepres,gravitynear,wefurth,modelfield
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
# from gensim import utils
# from gensim.models.doc2vec import LabeledSentence
# from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import os
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:


stop_words =  set(stopwords.words('english'))
stemmer = nltk.porter.PorterStemmer()
def review_to_wordlist(review, remove_stopwords=True):
    # Clean the text, with the option to remove stopwords.
    
    # Convert words to lower case and split them
    words = review.lower().split()

    # Optionally remove stop words (true by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    review_text = " ".join(words)

    # Clean the text
    review_text = re.sub(r"[^A-Za-z0-9(),!.?\'\`]", " ", review_text)
    review_text = re.sub(r"\'s", " 's ", review_text)
    review_text = re.sub(r"\'ve", " 've ", review_text)
    review_text = re.sub(r"n\'t", " 't ", review_text)
    review_text = re.sub(r"\'re", " 're ", review_text)
    review_text = re.sub(r"\'d", " 'd ", review_text)
    review_text = re.sub(r"\'ll", " 'll ", review_text)
    review_text = re.sub(r",", " ", review_text)
    review_text = re.sub(r"\.", " ", review_text)
    review_text = re.sub(r"!", " ", review_text)
    review_text = re.sub(r"\(", " ( ", review_text)
    review_text = re.sub(r"\)", " ) ", review_text)
    review_text = re.sub(r"\?", " ", review_text)
    review_text = re.sub(r"\s{2,}", " ", review_text)
    
    words = review_text.split()
    
    # Shorten words to their stems
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in words]
    
    review_text = " ".join(stemmed_words)
    
    # Return a list of words
    return(review_text)

def make_data(path):
  veriler = []
  pdfler = os.listdir(path)
  for pdf in pdfler:
    try:
        veri= {
            "isim":"",
        }
        veri.update({"isim" : pdf})
        dosya = open(path+pdf,"r")
        for metin in dosya:
            veri.update({"metin":metin})
        
        dosya.close()
        veriler.append(veri)
    except:
        print( "hata")
  return veriler

listpath_manu= "/content/drive/MyDrive/kodlamalar/okul_projesi/nlp/Dataset/manuscripts/"
manuscripts_veriler=make_data(listpath_manu)
for i in range(len(manuscripts_veriler)):
  metin = manuscripts_veriler[i]["metin"]
  metin = review_to_wordlist(metin,True)
  manuscripts_veriler[i].update({"metin":metin})

data_menu=[]
for i in range(len(manuscripts_veriler)):
    data_menu.append(manuscripts_veriler[i]["metin"])

tfvec = TfidfVectorizer()
tdf = tfvec.fit_transform(data_menu)
bow_menu = pd.DataFrame(tdf.toarray(), columns = tfvec.get_feature_names())
print(bow_menu)

listpath_reviewers= "/content/drive/MyDrive/kodlamalar/okul_projesi/nlp/Dataset/reviewers/"
reviewers_veriler=make_data(listpath_reviewers)
for i in range(len(reviewers_veriler)):
  metin = reviewers_veriler[i]["metin"]
  metin = review_to_wordlist(metin,True)
  reviewers_veriler[i].update({"metin":metin})

data_reviewers=[]
for i in range(len(reviewers_veriler)):
    data_reviewers.append(reviewers_veriler[i]["metin"])

tfvec = TfidfVectorizer()
tdf = tfvec.fit_transform(data_reviewers)
bow_reviewers = pd.DataFrame(tdf.toarray(), columns = tfvec.get_feature_names())
print(bow_reviewers)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


      00  0002  0009  002  0025  003  0034  01130  01296  01337  ...  zeroth  \
0    0.0   0.0   0.0  0.0   0.0  0.0   0.0    0.0    0.0    0.0  ...     0.0   
1    0.0   0.0   0.0  0.0   0.0  0.0   0.0    0.0    0.0    0.0  ...     0.0   
2    0.0   0.0   0.0  0.0   0.0  0.0   0.0    0.0    0.0    0.0  ...     0.0   
3    0.0   0.0   0.0  0.0   0.0  0.0   0.0    0.0    0.0    0.0  ...     0.0   
4    0.0   0.0   0.0  0.0   0.0  0.0   0.0    0.0    0.0    0.0  ...     0.0   
..   ...   ...   ...  ...   ...  ...   ...    ...    ...    ...  ...     ...   
700  0.0   0.0   0.0  0.0   0.0  0.0   0.0    0.0    0.0    0.0  ...     0.0   
701  0.0   0.0   0.0  0.0   0.0  0.0   0.0    0.0    0.0    0.0  ...     0.0   
702  0.0   0.0   0.0  0.0   0.0  0.0   0.0    0.0    0.0    0.0  ...     0.0   
703  0.0   0.0   0.0  0.0   0.0  0.0   0.0    0.0    0.0    0.0  ...     0.0   
704  0.0   0.0   0.0  0.0   0.0  0.0   0.0    0.0    0.0    0.0  ...     0.0   

     zerovalu  zeta  zinkernagel  zitte

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
